## Installing required libraries

In [1]:
%pip install torch transformers datasets wandb
%pip install setuptools
%pip install wheel
%pip install wandb
%pip install huggingface_hub[hf_xet]



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


#importing libralies

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import GPT2Tokenizer
from datasets import load_dataset
import wandb
from huggingface_hub import HfApi, HfFolder, Repository
import os
import random

c:\Users\hshakademie7\Desktop\INTOCODE WEITERBILDUNG\Generative_Al_project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
